# Загрузка данных

In [1]:
import numpy as np
import pandas as pd 
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/porn-detection-dataset/sample_submission.csv
/kaggle/input/porn-detection-dataset/train.csv
/kaggle/input/porn-detection-dataset/test.csv


In [2]:
train = pd.read_csv('/kaggle/input/porn-detection-dataset/train.csv')
test = pd.read_csv('/kaggle/input/porn-detection-dataset/test.csv')

# Проверка теории

Предположение заключается в том, что в датасете класс с меткой 0, относящийся к обычным сайтам, привалирует. Также оба класса (в частности классы с меткой 1) имеют определеный ряд самых часто употребляемых слов в url и title.

In [3]:
print(train['label'].value_counts())
print(train['label'].value_counts(normalize=True))

label
0    118594
1     16715
Name: count, dtype: int64
label
0    0.876468
1    0.123532
Name: proportion, dtype: float64


In [4]:
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

train['url'] = train['url'].fillna('').astype(str)
train['title'] = train['title'].fillna('').astype(str)

train['text'] = train['url'] + ' ' + train['title']

def get_top_words(text_series, n=30):
    all_text = ' '.join(text_series)

    # Токенизация текста
    words = word_tokenize(all_text.lower())

    # Загрузка стоп-слов
    stop_words = set(stopwords.words('english')) | set(stopwords.words('russian'))

    filtered_words = []
    for word in words:
        if word.isalpha() and word not in stop_words:
            filtered_words.append(word)

    word_counts = Counter(filtered_words)
    
    return word_counts.most_common(n)

top_words_0 = get_top_words(train[train['label'] == 0]['text'])
print("Топ-30 слов в классе 0 (не порнография):", top_words_0)
print("\n")

top_words_1 = get_top_words(train[train['label'] == 1]['text'])
print("Топ-30 слов в классе 1 (порнография):", top_words_1)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Топ-30 слов в классе 0 (не порнография): [('онлайн', 10013), ('бесплатно', 7219), ('скачать', 6976), ('купить', 6306), ('фото', 5343), ('страница', 5193), ('поиск', 5096), ('смотреть', 4224), ('читать', 3312), ('отзывы', 3248), ('цена', 3056), ('результатов', 2646), ('цены', 2472), ('продажа', 1982), ('работа', 1916), ('слушать', 1764), ('видео', 1755), ('тыс', 1691), ('года', 1667), ('качестве', 1656), ('россия', 1652), ('москве', 1593), ('хорошем', 1558), ('москва', 1552), ('мир', 1541), ('лет', 1513), ('руб', 1416), ('компании', 1401), ('xn', 1355), ('olx', 1354)]


Топ-30 слов в классе 1 (порнография): [('порно', 5163), ('porn', 3730), ('видео', 3000), ('sex', 1934), ('videos', 1877), ('онлайн', 1595), ('hd', 1435), ('video', 1425), ('смотреть', 1300), ('xxx', 1242), ('free', 1201), ('секс', 1148), ('бесплатно', 1017), ('фото', 883), ('скачать', 758), ('porno', 754), ('biqle', 734), ('girls', 712), ('movies', 710), ('страница', 650), ('page', 628), ('big', 620), ('daftsex', 614), (

In [5]:
test['text'] = test['url'] + ' ' + test['title']

# Тренировка модели

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# Векторизация текста
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train = tfidf_vectorizer.fit_transform(train['text'])
X_test = tfidf_vectorizer.transform(test['text'])
y_train = train['label']

In [7]:
X_train_train, X_train_test, y_train_train, y_train_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

logistic_regression = LogisticRegression(random_state=42, max_iter=500)
logistic_regression.fit(X_train_train, y_train_train)

LogisticRegression(max_iter=500, random_state=42)

# Оценка качества модели

In [8]:
y_pred_test = logistic_regression.predict(X_train_test) 

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, classification_report

accuracy = accuracy_score(y_train_test, y_pred_test)
print(f"Accuracy: {accuracy}")

precision = precision_score(y_train_test, y_pred_test)
print(f"Precision: {precision}")

recall = recall_score(y_train_test, y_pred_test)
print(f"Recall: {recall}")

f1 = f1_score(y_train_test, y_pred_test)
print(f"F1: {f1}")

Accuracy: 0.9871406400118247
Precision: 0.9908136482939632
Recall: 0.9041916167664671
F1: 0.9455228553537883


# Формирование submission.cvs

In [10]:
y_pred_test = logistic_regression.predict(X_test)

In [11]:
submission = pd.DataFrame({'ID': test['ID'], 'label': y_pred_test})
submission.to_csv('submission.csv', index=False)

print("Файл submission.csv успешно создан.")

Файл submission.csv успешно создан.
